Why LangChain?
- Modularity: Composable building blocks, tools and integrations for working with language models.
- Ease of Integration: Using Mistral instead of OpenAI is just a `Class` change away instead of having to learn an entirely new API.
- Enhanced Capabilities: LangChain Expression Language and Runnable interface.
- Community and Support: It's got the largest community.

In [101]:
from dotenv import load_dotenv

from langchain import hub
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import (
    create_tool_calling_agent,
    AgentExecutor,
)

from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
)

from langchain_community.vectorstores.faiss import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.chat_message_histories import (
    ChatMessageHistory,
    RedisChatMessageHistory,
)

load_dotenv()

True

##### 1.0 The Basics

In [2]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model = "gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} # because our prompt expects 'topic' as input
    | prompt # the pipes work just like bash scripting!
    | model # they're a small part called LangChain Expression Language
    | output_parser
)

In [3]:
# the most standard method.
chain.invoke("a parrot")

'Why did the parrot wear a raincoat? Because he wanted to be poly-unsaturated!'

Let's now look at the **Runnable Interface** (Stream, Invoke, Batch, Asynchronous).

In [4]:
# use LangChain's stream API (so we print out token by token)
for s in chain.stream("a parrot"):
    print(s, end="", flush=True)

Why did the parrot wear a raincoat?

Because he wanted to be polyunsaturated!

In [ ]:
# batch requests
chain.batch(['a parrot', 'the weather', 'Timothee Chalamet'])

['Why did the parrot wear a raincoat?\n\nBecause he wanted to be poly-unsaturated!',
 'Why did the weather go to therapy? It had too many mood swings!',
 'Why did Timothee Chalamet bring a ladder to the bar? \n\nBecause he heard the drinks were on a higher level!']

In [8]:
# asynchronous
async for s in chain.astream("a parrot"):
    print(s, end="", flush=True)

Why did the parrot wear a raincoat?

Because he wanted to be poly-"unsaturated"!

In [10]:
# asynchronous
await chain.ainvoke("a parrot")

'Why did the parrot wear a raincoat?\n\nBecause he wanted to be polyunsaturated!'

In [11]:
# asynchronous
await chain.abatch(['a parrot', 'the weather', 'Timothee Chalamet'])

['Why did the parrot wear a raincoat?\n\nBecause it wanted to be polyunsaturated!',
 'Why did the snowman bring a scarf to the beach?\nBecause he heard it was going to be a little chili!',
 'Why did Timothee Chalamet bring a ladder to the red carpet? \n\nBecause he heard the paparazzi were always looking for a higher angle to photograph him from!']

##### 2.0 Memory

In [14]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model = "gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [15]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="topic",
    history_messages_key='history',
)

In [17]:
with_message_history.invoke(
    {"topic": "bears"},
    {"configurable": {"session_id": "1"}}
)

'Why did the bear dissolve in water?\n\nBecause it was a polar bear!'

In [18]:
store

{'1': InMemoryChatMessageHistory(messages=[HumanMessage(content='bears', additional_kwargs={}, response_metadata={}), AIMessage(content='Why did the bear dissolve in water?\n\nBecause it was a polar bear!', additional_kwargs={}, response_metadata={})])}

In [ ]:
# since out session id is the same, it should make a joke about bears
with_message_history.invoke(
    {"topic": "Make a joke about an animal I just asked about"},
    {"configurable": {"session_id": "1"}}
)

'Why did the bear wear a fur coat in the summer? \n\nBecause he wanted to be bear-y stylish!'

In [22]:
print(store)

{'1': InMemoryChatMessageHistory(messages=[HumanMessage(content='bears', additional_kwargs={}, response_metadata={}), AIMessage(content='Why did the bear dissolve in water?\n\nBecause it was a polar bear!', additional_kwargs={}, response_metadata={}), HumanMessage(content='Make a joke about an animal I just asked about', additional_kwargs={}, response_metadata={}), AIMessage(content='Why did the bear dissolve in water? Because it was a polar bear!', additional_kwargs={}, response_metadata={}), HumanMessage(content='Make a joke about an animal I just asked about', additional_kwargs={}, response_metadata={}), AIMessage(content='Why did the bear wear a fur coat in the summer? \n\nBecause he wanted to be bear-y stylish!', additional_kwargs={}, response_metadata={})])}


Below is an alternative to an in-memory chat history storage solution. You can use Redis instead and still use `.invoke()`.

In [ ]:
# REDIS_URL = "redis://localhost:6379"

# def get_message_history(session_id: str) -> RedisChatMessageHistory:
#     return RedisChatMessageHistory(session_id, url=REDIS_URL)

# with_message_history = RunnableWithMessageHistory(
#     chain,
#     get_session_history=get_message_history,
#     input_messages_key="topic",
#     history_messages_key='history',
# )

##### 3.0 RAG (Retrieval Augmented Generation)

In [35]:
# Read PDF
pdf_reader = PyPDFLoader("../documents/Alexiou and Kartiyasa (2019).pdf")

pages = []
async for page in pdf_reader.alazy_load():
    pages.append(page)

In [47]:
pages[0].dict()

{'id': None,
 'metadata': {'source': '../documents/Alexiou and Kartiyasa (2019).pdf',
  'page': 0},
 'page_content': 'Received: 6 March 2019 Revised: 15 September 2019 Accepted: 27 November 2019\nDOI: 10.1111/boer.12226\nARTICLE\nDoes greater income inequality cause increased work\nhours? New evidence from high income economies\nConstantinos Alexiou Adimulya Kartiyasa\nCranfield University, College Road, UK\nCorrespondence\nConstantinos Alexiou, Cranfield University,\nCollege Road, Bedfordshire, MK430AL\nCranfield, UK.\nEmail: constantinos.alexiou@cranfield.ac.uk\nAdimulya Kartiyasa, Cranfield University,\nCollege Road, Bedfordshire, MK430AL\nCranfield, UK.\nEmail: A.Kartiyasa@cranfield.ac.uk\nAbstract\nWe explore the extent to which individual’s allocation of\ntime between labour and leisure is affected by the consump-\ntion standards of the rich. Utilizing a panel data method-\nology and panel Granger causality tests we investigate the\nrelationship between income inequality and work

In [36]:
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

{'source': '../documents/Alexiou and Kartiyasa (2019).pdf', 'page': 0}

Received: 6 March 2019 Revised: 15 September 2019 Accepted: 27 November 2019
DOI: 10.1111/boer.12226
ARTICLE
Does greater income inequality cause increased work
hours? New evidence from high income economies
Constantinos Alexiou Adimulya Kartiyasa
Cranfield University, College Road, UK
Correspondence
Constantinos Alexiou, Cranfield University,
College Road, Bedfordshire, MK430AL
Cranfield, UK.
Email: constantinos.alexiou@cranfield.ac.uk
Adimulya Kartiyasa, Cranfield University,
College Road, Bedfordshire, MK430AL
Cranfield, UK.
Email: A.Kartiyasa@cranfield.ac.uk
Abstract
We explore the extent to which individual’s allocation of
time between labour and leisure is affected by the consump-
tion standards of the rich. Utilizing a panel data method-
ology and panel Granger causality tests we investigate the
relationship between income inequality and work hours
for a cluster of 24 high-income OECD countries over the
peri

In [69]:
# create embeddings before loading into a vector store/knowledge base
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

# we'll use FAISS as our vector store.
vector_store = FAISS.from_documents(pages, embeddings)

retriever = vector_store.as_retriever(
    search_kwargs={
        "k": 3,
        # "score_threshold": 0.1
    }
)

In [70]:
# set a prompt template
template = """
Answer the question based only on the following context:

You are an assistant for question-answering tasks. Use the following pieces of
retrieved context to answer the question. If you don't know the answer, just
say you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context}

Answer:
"""

prompt = ChatPromptTemplate.from_template(template=template)

In [71]:
def format_docs(docs):
    """Function to only get `page_content` data from PyPDFLoader"""
    return "\n\n".join(doc.page_content for doc in docs)

In [72]:
llm = ChatOpenAI(model = "gpt-3.5-turbo")

rag_chain = (
    # RunnablePassThrough just gives the output as is
    RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [73]:
rag_chain.invoke("What did Bell and Freeman (2001) report?")

'Bell and Freeman (2001) reported that workers in the European Community prefer pay increases to decreases in hours, suggesting that current work hours are close to their ideal choice. This indicates that the relationship between inequality and work hours is influenced by individual preferences and perceived ideal working conditions. Additionally, similar results were found in a study by Böheim and Taylor (2004) using the British Household Panel Survey.'

##### 4.0 Tool Calling

###### 4.1 Tool Calling Only

For the below, **you must provide the docstring.** It's for the LLM to identify if it needs to use the tool or not.

In [76]:
@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
    - `city` (str): The name of the city where you want to check the weather.

    Returns:
    - (str): A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"

tools = [fake_weather_api]

In [77]:
llm = ChatOpenAI(model = "gpt-3.5-turbo")

llm_with_tools = llm.bind_tools(tools)

In [78]:
# you'll notice that content is empty!
result = llm_with_tools.invoke("How will the weather be in Munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NVK9urq9Sv5PsXt1YK0OzHe5', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 101, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6f0dddb8-774e-444c-bdfb-c717acf675d4-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_NVK9urq9Sv5PsXt1YK0OzHe5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 101, 'output_tokens': 17, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}

In [80]:
result.model_dump()

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'call_NVK9urq9Sv5PsXt1YK0OzHe5',
    'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'},
    'type': 'function'}],
  'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 17,
   'prompt_tokens': 101,
   'total_tokens': 118,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'finish_reason': 'tool_calls',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-6f0dddb8-774e-444c-bdfb-c717acf675d4-0',
 'example': False,
 'tool_calls': [{'name': 'fake_weather_api',
   'args': {'city': 'Munich'},
   'id': 'call_NVK9urq9Sv5PsXt1YK0OzHe5',
   'type': 'tool_call'}],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 101,
  'outp

In [82]:
messages = [
    HumanMessage(
        "How will the weather be in Munich today? I would like to eat outside if possible."
    )
]

llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [84]:
messages

[HumanMessage(content='How will the weather be in Munich today? I would like to eat outside if possible.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7tOMJ2uuBJODOqtJo8S5tybB', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 110, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4d879044-3466-424b-ae07-4cf35abaed6e-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_7tOMJ2uuBJODOqtJo8S5tybB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 

Here's the fix.

In [83]:
tool_mapping = {
    "fake_weather_api": fake_weather_api
}

tool_mapping

{'fake_weather_api': StructuredTool(name='fake_weather_api', description='Check the weather in a specified city.\n\nArgs:\n- `city` (str): The name of the city where you want to check the weather.\n\nReturns:\n- (str): A description of the current weather in the specified city.', args_schema=<class 'langchain_core.utils.pydantic.fake_weather_api'>, func=<function fake_weather_api at 0x7fa869ced8a0>)}

In [85]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [86]:
messages

[HumanMessage(content='How will the weather be in Munich today? I would like to eat outside if possible.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7tOMJ2uuBJODOqtJo8S5tybB', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 110, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4d879044-3466-424b-ae07-4cf35abaed6e-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_7tOMJ2uuBJODOqtJo8S5tybB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 

Finally!

In [88]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is sunny with a temperature of 22°C. It sounds like a perfect day to eat outside! Enjoy your meal!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 142, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e29b88de-28d1-49d2-a07d-0a5cc5472f19-0', usage_metadata={'input_tokens': 142, 'output_tokens': 30, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

###### 4.2 Agents

Agents are able to use tools autonomously and can reflect and use their own thoughts.

In [90]:
# create tool #1
@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
    - `city` (str): The name of the city where you want to check the weather.

    Returns:
    - (str): A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"

In [91]:
pages = []

for docs in [
    "../documents/Alexiou and Kartiyasa (2019).pdf",
    "../documents/Bell and Freeman (2001).pdf"
]:
    pdf_reader = PyPDFLoader(docs)

    async for page in pdf_reader.alazy_load():
        pages.append(page)

In [92]:
# create embeddings before loading into a vector store/knowledge base
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

# we'll use FAISS as our vector store.
vector_store = FAISS.from_documents(pages, embeddings)

retriever = vector_store.as_retriever(
    search_kwargs={
        "k": 3,
    }
)

In [93]:
# create tool #2
retriever_tool = create_retriever_tool(
    retriever,
    "academic_paper_retriever",
    "Get information about papers studying work hours and inequality",
)

In [94]:
tools = [fake_weather_api, retriever_tool]

In [96]:
# get a specialized prompt from LangChain Hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt

/home/faiq0913/Personal-Projects/chatbot-fastapi-rag-langchain-unstructured-pgvector/.venv/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [97]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [99]:
llm = ChatOpenAI(model = "gpt-3.5-turbo")

In [100]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [102]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
# use case where we ask a question about academic papers
agent_executor.invoke({"input": "What does Park and Bowles say about work hours?"})

{'input': 'What does Park and Bowles say about work hours?',
 'output': 'Park and Bowles, in their study, discuss the relationship between income inequality and work hours. They suggest that pay inequality serves as an indicator of incentives in a labor supply equation, with wider earnings distribution leading to greater work hours and effort by workers to improve their position in the earnings distribution. The study highlights empirical evidence from Lazear and Rosen\'s tournament model, indicating that wage inequality influences how much individuals are willing to work. Additionally, the study references Veblen\'s and Duesenberry\'s empirical studies, showing that consumption patterns and income inequality significantly impact individual consumption and working patterns.\n\nFurthermore, the study by Bell and Freeman demonstrates a positive relationship between earnings inequality and work hours in the US and Germany. It suggests that hours worked can affect future wages and promotio

In [ ]:
# use case where we ask a question about the weather
agent_executor.invoke({"input": "What's the weather in Kuala Lumpur?"})

{'input': "What's the weather in Kuala Lumpur?",
 'output': 'The current weather in Kuala Lumpur is sunny with a temperature of 22°C.'}